# Variants-Experiment 3.1, 3.2, 3.3.1, 3.3.2. Adapted from HiCoTiNe v2.1
## Concatenated hidden representations.
### No raw input¶

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from keras import backend as K

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
SUB_REGION_SCALE = 14
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6 , 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]
ALPHA = 0.1

#dataset specific parameters
NUM_CLASS = 5

## Data Pre-Processing

In [5]:
with open('../input/hicotine1-layer1-biodiversity/fyp/CIFAR-10.pickle', 'rb') as f:
    X_train_cropped_list, y_train_one_hot, X_val_cropped_list, y_val_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list = pickle.load(f)

## HiCo Layer 1

In [6]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [7]:
for i in range(NUM_TN[0]):
  ensemble[i][0].load_weights('../input/hicotine1-layer1-biodiversity/fyp/CIFAR-10_' + str(i) + '.h5')

In [8]:
hico_layers = []
hico_layers.append(ensemble)

In [9]:
del hico_layers[1:]

## HiCo Layer 2+

In [10]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[1].output))
      input.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input = np.array(input)
    input = np.concatenate(input, axis=1)
    previous_layer_input = input
    return previous_layer_input

In [11]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(rn.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=64*NUM_CONNECTION[i]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_train_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list)))
    X_train_input = np.array(X_train_input)
    X_train_input = np.concatenate(X_train_input, axis=1)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5, batch_size=128)

Layer 1
HICO LENGTH
2
Model 0
Epoch 1/5
28/28 [==============================] - 1s 22ms/step - loss: 1.5587 - accuracy: 0.2993 - val_loss: 1.2995 - val_accuracy: 0.4226
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2395 - accuracy: 0.4446 - val_loss: 1.2655 - val_accuracy: 0.4527
Epoch 3/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2379 - accuracy: 0.4746 - val_loss: 1.2649 - val_accuracy: 0.4457
Epoch 4/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2068 - accuracy: 0.4837 - val_loss: 1.2479 - val_accuracy: 0.4642
Epoch 5/5
28/28 [==============================] - 0s 4ms/step - loss: 1.1540 - accuracy: 0.4939 - val_loss: 1.2548 - val_accuracy: 0.4550
Model 1
Epoch 1/5
28/28 [==============================] - 1s 9ms/step - loss: 1.5465 - accuracy: 0.2989 - val_loss: 1.3044 - val_accuracy: 0.3972
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2366 - accuracy: 0.4505 - val_loss: 1.2675 - val_accur

# Model Evaluation

In [12]:
y_test = np.argmax(y_test_one_hot, axis=1)

### HiCoTiNe1 v3.1
cc1. Mode. Compute classifications of all tiny networks, in all HiCo layers, and take the mode as the final classification.

In [13]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = np.argmax(hico_layers[0][j][0].predict(X_test_cropped_list[j]), axis=1)
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    y_pred = np.argmax(hico_layers[i][j][0].predict(X_test_input), axis=1)
    y_pred_list.append(y_pred)

# HiCo voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0))
y_pred_list = stats.mode(y_pred_list, axis=1)[0]
y_pred_list = np.squeeze(y_pred_list)

In [14]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.4665127020785219

### HiCoTiNe v3.2
cc2. Sum the logits of all TNs. Take the max logit as the final classification.

In [15]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[j])
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    y_pred = hico_layers[i][j][0].predict(X_test_input)
    y_pred_list.append(y_pred)

# HiCo voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
y_pred_list = np.argmax(np.sum(y_pred_list, axis=1), axis=1)

In [16]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.46882217090069284

### HiCoTiNe v3.3.1
cc3. Weighted function. Compute classifications (votes) of all tiny networks, in all HiCo layers. Compute weight of each classifier/vote. Compute weighted sum of votes. Weights can be computed in different ways, e.g.:
- cc3w1. The weight of each classifier/TN is inversely proportional to the number of TNs in its HiCo layer, relative to the HiCo layer with the most TNs. E.g. if n is the number of TNs in the layer with the most TNs, then compute v/n for each layer, where v is the number of TNs in each layer; then compute the weight of each layer as (1+alpha)-(v/n), where alpha is some small threshold; and finally define the weight of each TN as the weight of its layer.


In [17]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[j])
  y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[0] / max(NUM_TN)))
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    y_pred = hico_layers[i][j][0].predict(X_test_input)
    y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[i] / max(NUM_TN)))
    y_pred_list.append(y_pred)


# HiCo voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
y_pred_list = np.argmax(np.sum(y_pred_list, axis=1), axis=1)

In [18]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.47344110854503463

### HiCoTiNe v3.3.2
cc3. Weighted function. Compute classifications (votes) of all tiny networks, in all HiCo layers. Compute weight of each classifier/vote. Compute weighted sum of votes. Weights can be computed in different ways, e.g.:
- cc3w2. The weight of each classifier is directly proportional to the depth of the HiCo layer within which it resides. 


In [19]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[j])
  y_pred = y_pred * (1 / NUM_HICO_LAYER)
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    y_pred = hico_layers[i][j][0].predict(X_test_input)
    y_pred = y_pred * (i / NUM_HICO_LAYER)
    y_pred_list.append(y_pred)

# HiCo voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
y_pred_list = np.argmax(np.sum(y_pred_list, axis=1), axis=1)

In [20]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.4780600461893764